In [3]:
import math
import torch
import cv2
import mediapipe as mp
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,
                    model_complexity=1,
                    smooth_landmarks=True,
                    # enable_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

I0000 00:00:1716385761.869426  125937 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1716385761.871203  126121 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.7), renderer: Mesa Intel(R) Arc(tm) Graphics (MTL)


W0000 00:00:1716385761.931800  126102 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716385761.949845  126103 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
def point(landmark):
    return torch.tensor([landmark.x,-landmark.y])
def angle(vec1, vec2):
    cross_z = vec1[0] * vec2[1] - vec1[1] * vec2[0]
    dot_product = torch.dot(vec1, vec2)
    magnitude_vec1 = torch.norm(vec1)
    magnitude_vec2 = torch.norm(vec2)
    sin_theta = cross_z / (magnitude_vec1 * magnitude_vec2)
    cos_theta = dot_product / (magnitude_vec1 * magnitude_vec2)
    angle_rad = torch.atan2(sin_theta, cos_theta)
    angle_deg = torch.rad2deg(angle_rad) % 360
    
    return angle_deg.item()

In [5]:
Esh = 20
Elbow = 20
Bsh = 30
Blbow = 20

def detectionBE(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img)
    rs = point(results.pose_landmarks.landmark[12])
    ls = point(results.pose_landmarks.landmark[11])
    re = point(results.pose_landmarks.landmark[14])
    le = point(results.pose_landmarks.landmark[13])
    rw = point(results.pose_landmarks.landmark[16])
    lw = point(results.pose_landmarks.landmark[15])
    rh = point(results.pose_landmarks.landmark[24])
    lh = point(results.pose_landmarks.landmark[23])
    rShoulder = angle(rs - re, rs - rh)
    lShoulder = angle(ls - lh, ls - le)
    rElow = angle(re - rw, re - rs)
    lElow = angle(le - ls, le - lw)
    #print(rShoulder,lShoulder, rElow, lElow)
    #print(rs - re, rs - rh)
    isEnd = (
        0 <= rShoulder <= Esh
        and 0 <= lShoulder <= Esh
        and (180 - Elbow) < rElow < (180 + Elbow)
        and (180 - Elbow) < lElow < (180 + Elbow)
    )
    isBegin = (
        90 <= rShoulder <= 90 + Bsh
        and 90 <= lShoulder <= 90 + Bsh
        and (180 - Blbow) < rElow < (180 + Blbow)
        and (180 - Blbow) < lElow < (180 + Blbow)
    )
    return isBegin,isEnd;

In [9]:
img = cv2.imread("2.jpg")
print(detectionBE(img))

(False, True)


### mp_pose.Pose()其参数：

1. static_image_mode：静态图像还是连续帧视频;

2. model_complexity：人体姿态估计模型,0表示速度最快,精度最低（三者之中）,1表示速度中间，精度中间（三者之中),2表示速度最慢，精度最高（三者之中）;

3. smooth_landmarks：是否平滑关键点；

4. enable_segmentation：是否对人体进行抠图;

5. min_detection_confidence：检测置信度阈值；

6. min_tracking_confidence：各帧之间跟踪置信度阈值；